# Finde ein effizientes Programm um eine Floyd Rose Gitarre zu stimmen

## Einleitung

Floyd Rose Gitarren sind eine spezielle Art von E-Gitarren, die eine spezielle Art von Brücke verwenden, die als Floyd Rose Tremolo bekannt ist. Diese Brücke ermöglicht es dem Gitarristen, die Tonhöhe der Saiten zu ändern, indem er einen Hebel betätigt, der an der Brücke befestigt ist. Dies ermöglicht es dem Gitarristen, spezielle Effekte wie Vibrato und Divebombs zu erzeugen. Es erschwert jedoch auch das Stimmen der Gitarre erheblich, da bei dem Stimmen einer Saite die anderen Saiten ebenfalls verstimmt werden können, da die Spannung der Saiten auf der Brücke verteilt ist. Ein Grund dafür ist das die Saiten an Federn aufgehängt sind. Dies kann dazu führen, dass das Stimmen einer Floyd Rose Gitarre sehr zeitaufwändig ist.

In dieser Arbeit stellen wir zunächst ein Experiment auf, welches uns informationen über das Verhalten der Saiten gibt, wenn wir eine Saite stimmen. Mit diesen Informationen leiten wir ein mathematisches Modell ab, welches uns erlaubt, die Saiten einer Floyd Rose Gitarre effizient zu stimmen. Es soll berechnet werden welche Saiten in welcher Reihenfolge zu welcher Frequenz gestimmt werden müssen, um die Gitarre möglichst schnell und genau zu stimmen.

Auf dem Weg zu diesem Ziel werden wir ein Programm erstellen, welches uns erlaubt die Frequenz von Saiten zu messen. Die Messdaten von dem Experiment werden Visualisiert und anhand dessen wird das mathematische Modell erstellt.

Anschließend wird ein Program erstellt welches berechnet, welche Saiten in welcher Reihenfolge zu welcher Frequenz gestimmt werden müssen, um die Gitarre möglichst schnell und genau zu stimmen.

## Experiment

Um die Saiten einer Floyd Rose Gitarre effizient zu stimmen, müssen wir zunächst ein Experiment durchführen, welches uns Informationen über das Verhalten der Saiten gibt, wenn wir eine Saite stimmen.

Dazu wird zunächst jede Saite in eine Ausgangsposition gebracht. Eine Standard Gitarren Stimmung ist EADGBE. Die Saiten werden so gestimmt, dass sie die Frequenzen E2, A2, D3, G3, B3 und E4 haben. Es handelt sich hierbei um die gleich temperierte Stimmung[1].
Die Saiten werden demnach die Frequenzen haben:\
E2 = 82.41 Hz\
A2 = 110.00 Hz\
D3 = 146.83 Hz\
G3 = 196.00 Hz\
B3 = 246.94 Hz\
E4 = 329.63 Hz

Da eine Gitarre sich nie perfekt stimmen lässt, vor allem keine Floyd Rose Gitarre, wird die Ausgangsfrequenz der Saiten gemessen und aufgezeichnet. Die Saiten werden aber zu Beginn den optimalen Frequenzen angenähert.

Dann wird jede Saite um ein beliebiges delta verstimmt. Das Delta wird so gewählt dass die Saite sich deutlich hörbar verstimmt. Es gibt also verschiedene willkürliche gewählte deltas. Auch weil es nicht möglich ist, die Saiten exakt zu verstimmen.

Jede Saite wird um 4 Schritte jeweils nach oben und nach unten verstimmt. Für jeden Schritt wird die Frequenz der anderen Saiten gemessen und aufgezeichnet.

Schritte in beide Richtungen = 4

Das Ziel des Experiments ist zu beobachten ob das System
- linear ist
- elastisch ist

Linear ist es, wenn die Verstimmung einer Saite die Frequenz der anderen Saiten linear beeinflusst. Elastisch ist es, wenn die Frequenz einer Saite wieder in ihren Ausgangszustand zurückkehrt, wenn die Verstimmung aufgehoben wird.
Dazu werden nach allen Verstimmungsschritten verglichen ob die Frequenz der Saiten wieder in den Ausgangszustand zurückkehrt.

Die erkenntnisse bestimmen später das mathematische Modell.


### Materialien
- Floyd Rose Gitarre
- Frequenz Messgerät
- Visualisierungssoftware
- Audio Aufnahme Software

Die Floyd Rose Gitarre ist verfügbar. Das Frequenz Messgerät wird ein Python Programm sein, welches die Frequenz der Saiten misst. Die Visualisierungssoftware wird ebenfalls ein Python Programm sein, welches die Messdaten visualisiert.

#### Frequenz Messgerät

Gegeben sei eine Liste von Samples. Diese Liste wird mit Hilfe der Fourier Transformation in den Frequenzbereich transformiert. Die Frequenz mit der höchsten Amplitude wird als die Frequenz der Saite angenommen.


In [1]:
from scipy.fftpack import fft
from scipy.io import wavfile  # Import the wavfile API
import numpy as np

def get_samples(filepath):
    fs, data = wavfile.read(filepath)  # Load the data
    if len(data.shape) > 1:  # Check if stereo
        data = data.mean(axis=1)  # Convert to mono by averaging channels
    return data, fs

def fourier_transform(samples):
    spectrum = fft(samples)
    half = len(spectrum) // 2  # Take the first half of the spectrum
    return spectrum[:half]

def get_peak_frequency(spectrum, samplerate):
    # Get the magnitude of the spectrum
    magnitude = np.abs(spectrum)
    # Find the index of the peak frequency
    peak_idx = np.argmax(magnitude)
    # Map the index to a frequency value
    frequency = peak_idx * (samplerate / (2 * len(spectrum)))
    return frequency

def get_frequency_from_file(filepath):
    samples, samplerate = get_samples(filepath)
    spectrum = fourier_transform(samples)
    return get_peak_frequency(spectrum, samplerate)

#### Cent Messgerät
Da wir nun die Frequenz Messen können, müssen wir den Unterschied zweier Frequenzen in Cent umwandeln. Dazu wird die folgende Funktion erstellt.

In [2]:
def frequency_difference_to_cent(f1,f2):
    return 1200 * np.log2(f1/f2) # 1200 Cent entsprechen einem Halbton


#### Visualisierungssoftware

Mit hilfe von matplotlib wird eine Visualisierung erstellt. Die Visualisierung zeigt die Frequenz der Saiten in Abhängigkeit von der Verstimmung.
Die daten zum Visualisieren werden wie folgt strukturiert:

```
{
    'E2': {
        'E2': array([-4,...,4]),
        'D3': array([record_-4,record_-3,...,record_4]),
        ...,
        'E4': array([records_per_step,...])
    },
    ...
    'E4': {
        'E2': array([records_per_step,...]),
        'A2': array([records_per_step,...]),
        ...,
        'E4': array([-4,...,4])
    }
}
```

Das folgende Programm ermöglicht es uns, die Daten zu visualisieren. Es wird eine Checkbox für jede Saite erstellt, die es uns ermöglicht, die Daten für jede Saite einzeln anzuzeigen. Die Visualisierung zeigt die Auswirkungen der Verstimmung einer Saite auf die Frequenz der anderen Saiten. Da interessant ist, ob die Verstimmung Logarithmisch ist, wird die änderung er Frequenz in Cent und in Hz dargestellt.

In [3]:
import matplotlib.pyplot as plt
from ipywidgets import  HBox, VBox, interactive, Layout, Checkbox, fixed
import random

nStrings = 6
steps = 4
step_range = range(-steps, steps+1) # -10 bis 10
strings = ["E2", "A2", "D3", "G3", "B3", "E4"]
# Verstimmung jeder anderen seite für jede Saite für alle Verstimmungsschritte
example_dataset = { # Dummy Data
    string: {other_string: np.array([1000+( -_*random.random()if other_string != string else _ )for _ in step_range]) for other_string in strings}
    for string in strings
}


def visualisation(df,string,scale,**args):
    for other_string in strings:
        if other_string in args.keys() and args[other_string]:
            plt.plot(step_range, df[string][other_string], label=other_string)
    plt.title("Impact on String when detuning String "+string)
    plt.xlabel(f"Detuning of String {string} in Cent")
    plt.ylabel(f"Frequency of other Strings in Cent")
    plt.yscale(scale)
    #plt.xscale(scale)
    plt.legend()


def visualize_all(data,scale):
    widget_list = []

    for string in strings:
        checkboxes = {string: Checkbox(value=True,label=string,indent=False) for string in strings}
        widget = interactive(visualisation,df=fixed(data),string=fixed(string),scale=fixed(scale),**checkboxes)
        controls = HBox(widget.children[:-1]) # Horizontale Box für die Checkboxes
        output = widget.children[-1]
        w=VBox([controls, output], layout=Layout(margin="10px"))
        widget_list.append(w)
    row1 = HBox(widget_list[:3])
    row2 = HBox(widget_list[3:])
    output = VBox([row1, row2])
    display(output)



visualize_all(example_dataset,scale="log")
visualize_all(example_dataset,scale="linear")


### Durchführung

In der Live Demo wird das Experiment durchgeführt. Die Frequenz der Saiten wird gemessen und die Daten werden visualisiert.
Hier werden nun vorab aufgenommene Daten verwendet. Die Audio Dateien sind im Ordner `audio` zu finden.
Die Ordnerstruktur ist wie folgt:

    ```
    audio/<variable_saite>/<beeinflusste_saite>/<verstimmungsschritt>.wav

    audio
    ├── E2 # Ändernde Saite
      ├── E2 # Beinflusste Saite
        ├─── -4.wav
        ├─── ...
        ├─── 4.wav
      ├─── ...
      ├─── E4
    ├── ...
    ├── E4

#### Setup
Die Gitarre wird per Klinken Kabel an eine Audio Karte angeschlossen und mit einer Abtastrate von 44100 Hz aufgenommen.
Das Bild Zeigt wie die Gitarre positioniert war, damit der Gitarrenhals sich nicht verzieht.

<div>
    <img src="assets/setup.jpg" width="500"/>
</div>

Das digitale Setup ist wie folgt. Es wird die Audio Workstation "Cubase" verwendet. Die Aufnahme erfolgt in Mono. Die Aufnahme wird in 16 Bit und 44100 Hz aufgenommen.

<div>
    <img src="assets/digital_setup.png" width="500"/>
    Digitales Setup
</div>

Es wird ein Integriertes Stimmgerät verwendet, um die Saiten zu stimmen und wieder zurück zu stimmen.

Jeder Aufnahmeblock enthält den Klang aller Saiten der Gitarre. Also klingend E2, A2, D3, G3, B3 und E4. Die Saiten werden in der Reihenfolge E2, A2, D3, G3, B3 und E4 gespielt.

Die Farbe der Aufnahme Blöcke ist eine Visualisierung dafür wie stark die Saiten verstimmt wurden und in welche Richtung. Je röter die Farbe, desto stärker die Verstimmung zu tieferen Frequenzen. Je blauer die Farbe, desto stärker die Verstimmung zu höheren Frequenzen.

Jede Zeile Widmet sich einer Saite und einer Verstimmungsrichtung. Von oben nach unten ist die Reihenfolge: E2 nach oben, E2 nach unten, A2 nach oben, A2 nach unten, ..., E4 nach oben, E4 nach unten.



<div>
<img src="assets/cutted_audio.png" width="500"/>
Geschnittene Audio Blöcke
</div>

Anschließen werden die Einzelnen Audioblöcke so zu geschnitten dass neue Audio Blöcke entstehen wo der Transient der Saite abgeschnitten wurde und das Ende.

### Clean Up

<div>
    <img src="assets/need_to_clean_data.png" width="500"/>
    Der Screenshot Zeigt, dass nicht die Grundfrequenz als solche erkannt wird, sondern teilweise Obertöne als die Grundfrequenz erkannt werden. <br>
    Deshalb wird für jede Saite ein Bandpass Filter erstellt, der die Obertöne reduziert.
</div>

Banpass Filter:
<div>
    <img src="assets/bandpass_e2.png" width="500"/>
    Bandpass für E2
<div>
<div>
    <img src="assets/bandpass_a2.png" width="500"/>
    Bandpass für A2
</div>
<div>
    <img src="assets/bandpass_d3.png" width="500"/>
    Bandpass für D3
</div>
<div>
    <img src="assets/bandpass_g3.png" width="500"/>
    Bandpass für G3
</div>
<div>
    <img src="assets/bandpass_b3.png" width="500"/>
    Bandpass für B3
</div>
<div>
    <img src="assets/bandpass_e4.png" width="500"/>
    Bandpass für E4
</div>

Mit hilfe eines Scriptes werden die Audio Dateien in die Ordnerstruktur gespiechert.




In [9]:
# Experiment
measured_data = {}
filenames =  [f"{i}.wav" for i in step_range]

for changing_string in strings:
    for impacted_string in strings:
        #f0 = get_frequency_from_file(f"audio/{changing_string}/{impacted_string}/0.wav")

        for filename in filenames:
            frequency = get_frequency_from_file(f"audio/{changing_string}/{impacted_string}/{filename}")

            print(f"Saite: {changing_string}, Beeinflusste Saite: {impacted_string}, Verstimmungsschritt: {filename}, Frequenz: {frequency}")

            if changing_string not in measured_data:
                measured_data[changing_string] = {}

            if impacted_string not in measured_data[changing_string]:
                measured_data[changing_string][impacted_string] = []

            measured_data[changing_string][impacted_string].append(frequency)

C:\Users\guita\AppData\Local\Temp\ipykernel_26320\3270566540.py:6: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, data = wavfile.read(filepath)  # Load the data


Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: -4.wav, Frequenz: 57.56155443488579
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: -3.wav, Frequenz: 67.89055643973762
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: -2.wav, Frequenz: 74.80436177036562
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: -1.wav, Frequenz: 78.13278008298755
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 0.wav, Frequenz: 82.29854689564068
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 1.wav, Frequenz: 84.9931433230792
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 2.wav, Frequenz: 87.45281541913823
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 3.wav, Frequenz: 92.93188805334641
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 4.wav, Frequenz: 97.02425718679493
Saite: E2, Beeinflusste Saite: A2, Verstimmungsschritt: -4.wav, Frequenz: 114.39652438637661
Saite: E2, Beeinflusste Saite: A2, Verstimmungsschritt: -3.wav, Frequenz: 112.8823800

## 3. Visualisierung

In [10]:
visualize_all(measured_data,scale="linear")
visualize_all(measured_data,scale="log")

### 3.1 Clean up
Die Fourier Transformation ist nicht perfekt. Teilweise werden Obertöne als die Grundfrequenz erkannt. Um die Daten zu bereinigen werden die Peaks der Fourier Transformation manuell ausgewählt.

In [8]:
for changing_string in strings:
    print("______________________")
    for impacted_string in strings:
        print(changing_string, impacted_string)
        recordings = np.array(measured_data[changing_string][impacted_string])
        print("Recordings: ",recordings)



______________________
E2 E2
Recordings:  [57.56155443 67.89055644 74.80436177 78.13278008 82.2985469  84.99314332
 87.45281542 92.93188805 97.02425719]
E2 A2
Recordings:  [114.39652439 112.88238009 111.91015478 111.2360303  110.4554986
 110.0843391  109.62357955 108.54113439 107.76268682]
E2 D3
Recordings:  [152.30757423 150.51958433 149.26464051 148.66658078 147.60843857
 147.02649126 146.47011601 145.10846843 144.06110334]
E2 G3
Recordings:  [209.06962694 205.11627907 202.02724488 200.64114498 199.29850595
 197.67273414 196.76667813 194.21417793 191.93250582]
E2 B3
Recordings:  [255.99868567 253.23518363 251.14926016 250.04758053 248.76361443
 248.02926408 247.06691109 245.05839114 243.48127389]
E2 E4
Recordings:  [336.35758946 334.12261555 332.71604938 331.91452236 331.08500059
 330.42385348 329.87049157 328.53481219 327.54200463]
______________________
A2 E2
Recordings:  [86.48760331 84.83636247 83.91514753 83.10727655 82.38878673 81.98789257
 81.20257786 80.25881697 79.44068748]


### 4.1 Mathematisches Modell

Die Messdaten zeigen dass das System linear ist. Das bedeutet, dass die Verstimmung einer Saite die Frequenz der anderen Saiten linear beeinflusst. Das System ist auch elastisch, was bedeutet, dass die Frequenz einer Saite wieder in ihren Ausgangszustand zurückkehrt, wenn die Verstimmung aufgehoben wird.

Wir können das System als ein lineares Gleichungssystem beschreiben. Die Frequenz der Saiten kann als Vektor dargestellt werden. Die Verstimmung der Saiten kann als Matrix dargestellt werden. Die Verstimmung der Saiten ist eine lineare Transformation der Frequenz der Saiten.

$$
\vec{f_0} = \begin{pmatrix} {f_{E2}} \\ {f_{A2}} \\ {f_{D3}} \\ {f_{G3}} \\ {f_{B3}} \\ {f_{e4}} \end{pmatrix},
C = \begin{bmatrix} {1} & { c_{12} } & {c_{13}} & {c_{14}} & {c_{15}} & {c_{16}} \\ {c_{21}} & {1} & {c_{23}} & {c_{24}} & {c_{25}} & {c_{26}} \\ {c_{31}} & {c_{32}} & {1} & {c_{34}} & {c_{35}} & {c_{36}} \\ {c_{41}} & {c_{42}} & {c_{43}} & {1} & {c_{45}} & {c_{46}} \\ {c_{51}} & {c_{52}} & {c_{53}} & {c_{54}} & {1} & {c_{56}} \\ {c_{61}} & {c_{62}} & {c_{63}} & {c_{64}} & {c_{65}} & {1} \end{bmatrix},
\vec{g} = \begin{pmatrix} {\hat{f}_{E2}} \\ {\hat{f}_{A2}} \\ {\hat{f}_{D3}} \\ {\hat{f}_{G3}} \\ {\hat{f}_{B3}} \\ {\hat{f}_{e4}} \end{pmatrix}
$$
\
$\vec{f_0}$ sind die Frequenzen der Saiten in der Ausgangsposition.\
$C$ ist die Verstimmungsmatrix. Die Elemente $c_{ij}$ sind die Verstimmungsfaktoren der Saite $i$, wenn die Saite $j$ um 1 Schritt (20 Cent) verstimmt wird.\
$\vec{g}$ sind die Frequenzen der Saiten nach der Verstimmung.\
$\vec{g}$ wollen wir im allgemeinen als Zielfrequenzen erreichen.

Die Annahme ist, es gibt eine Verstimmung $\vec{\delta}$ der Saiten, die die Frequenzen der Saiten in $\vec{g}$ erreicht, wenn sie dem System $C$ unterworfen wird.

$$
\vec{g} = add\_cent\_to\_freq(C \cdot \vec{\delta}, \vec{f_0})
$$

$$
\vec{\delta} = cent(\vec{f_0}-\vec{g}) \cdot C^{-1}
$$

$C^{-1}$ ist die Inverse der Verstimmungsmatrix.\
$cent$ ist die Funktion, die die Frequenzdifferenz in Cent umwandelt.\
$add\_cent\_to\_freq$ ist die Funktion, die die Verstimmung in Cent zu den Frequenzen der Saiten addiert.

### 5. Programmierung
Im folgendem werden die Funktionen `cent`, `add_cent_to_freq` und `find_coefficients` implementiert.

#### 5.1 Funktionen




In [ ]:
def cent(f1,f2):
    return 1200 * np.log2(f1/f2) # 1200 Cent entsprechen einer Oktave

def add_cent_to_freq(cent,freq):
    return freq * 2 ** (cent/1200)/12

def find_coefficients(f0, g, C):
    return (g-f0) @ np.linalg.inv(C)


## Literatur
[1]Gleich temperierte Stimmung: https://de.wikipedia.org/wiki/Gleichstufige_Stimmung \
[2]Cent: https://de.wikipedia.org/wiki/Cent_(Musik)

